## Read Vertical Speed

Uses model to open and interpret vertical speed readings

TODO: Make this work on video, output GIF of read values.

## Imports

In [1]:
import cv2
import numpy as np
import tensorflow as tf

## Read Vertical Speed

Currently only works on a single file

In [2]:
model = tf.keras.models.load_model('../../models/regenerated_vertical_speed_model_6_epoch.h5')

def preprocess_frame(frame):
    frame = frame / 255.0  # Normalize pixel values to [0, 1]
    frame = np.expand_dims(frame, axis=0)  # Add batch dimension
    return frame


def predict_vertical_speed(frame):
    preprocessed_image = preprocess_frame(frame)

    predictions = model.predict(preprocessed_image)

    print(predictions)

    predicted_speed = np.argmax(predictions, axis=1)

    return (predicted_speed[0] * .25) - 6


In [3]:
# Hacky means of making sure we don't go into infinite loop
def exit_application():
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.waitKey(1)
    cv2.waitKey(1)

In [4]:
input_file_path = '../../media/input_video/static_camera/daytime_A10C_original_validation.mp4'

# Coordinates for vertical speed indicator
start_x = 1062
start_y = 650

width = 125
height = 120

overlay_x = 200
overlay_y = 200

scale_factor = 3

cap = cv2.VideoCapture(input_file_path)

while True:
    ret, frame = cap.read()

    if not ret:
        break  # Break the loop if no more frames are available

    x, y, w, h = (start_x, start_y, width, height)
    cropped_frame = frame[y:y+h, x:x+w]

    resized_cropped_frame = cv2.resize(cropped_frame, None, fx=scale_factor, fy=scale_factor)

    frame[overlay_y:overlay_y+(h*scale_factor), overlay_x:overlay_x+(w*scale_factor)] = resized_cropped_frame

    vertical_speed = predict_vertical_speed(cropped_frame)

    print(vertical_speed)

    cv2.putText(frame, f'Vertical Speed: {vertical_speed}', (150, 200), cv2.FONT_HERSHEY_PLAIN, 4.0, (0, 255, 0), 2)


    # Display the current frame
    cv2.imshow('Frame', frame)

    # Wait for user input (e.g., a key press)
    key = cv2.waitKey(0)

    # Check if the user pressed the 'q' key to exit
    if key == ord('q'):
        break

# Release the video capture object and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()
exit_application()

1/1 [==============================] - 0s 43ms/step
[[9.95314002e-01 3.07816197e-03 2.04950065e-05 9.69508474e-05
  1.06803014e-07 1.43632448e-07 2.23069545e-08 1.24226485e-09
  1.25156916e-12 1.57019450e-11 5.47111245e-09 1.10460929e-08
  2.30934449e-09 4.07378886e-09 3.33980417e-08 6.76749153e-07
  3.67482187e-08 3.43320389e-05 5.93208078e-08 5.27256294e-11
  9.45177874e-08 3.42013550e-07 6.54255594e-10 1.81966513e-08
  4.40235563e-05 4.33666010e-08 6.27113972e-10 7.47447757e-08
  1.43678458e-09 1.40405652e-08 2.37594395e-05 1.01798403e-09
  7.51974731e-05 8.65209313e-06 2.46257024e-08 1.29058375e-09
  4.69478845e-10 2.72761372e-06 2.95839909e-05 1.98305133e-06
  4.56039634e-05 1.42239605e-05 2.48555618e-04 1.48984654e-07
  7.24731808e-06 2.31240122e-07 2.70265876e-09 1.76255344e-04
  7.76058820e-04]]
-6.0
1/1 [==============================] - 0s 13ms/step
[[9.9542761e-01 2.9888442e-03 1.9960584e-05 9.7594151e-05 1.0313978e-07
  1.3773517e-07 2.1547315e-08 1.2409335e-09 1.2261585e-1